In [1]:
# Install dependencies
!pip install -q boto3 llama-index-llms-bedrock llama-index-embeddings-bedrock llama-index-graph-stores-neptune llama-index

In [2]:
# Import features
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import (
  StorageContext,
  SimpleDirectoryReader,
  KnowledgeGraphIndex,
  Settings
)
from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.graph_stores.neptune import NeptuneDatabaseGraphStore
from IPython.display import Markdown, display

In [3]:
# For the variables that follow, update the region, give Neptune notebook instance IAM role permission to invoke Bedrock using the two models specified below, and update the neptune_endpoint based on your Neptune database endpoint; also, check on AWS Bedrock console and make sure you have access to the Bedrock models
region_name = "us-east-1"
llmodel = "mistral.mistral-7b-instruct-v0:2"
embed_model = "amazon.titan-embed-text-v1"
neptune_endpoint = "" # add neptune_endpoint

In [10]:
import boto3
import os
# Initialize boto3 client with bedrock-runtime to access bedrock related feature
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name="us-east-1",
    aws_access_key_id="", # add aws_access_key_id
    aws_secret_access_key="", # add aws_secret_access_key
)

In [11]:
# Define LLM
llm = Bedrock(
  model=llmodel,
  region_name=region_name,
  client=bedrock_client,
)
embed_model = BedrockEmbedding(
    model=embed_model,
    client=bedrock_client
)
# Settings.llm = llm
# Settings.chunk_size = 512
# Settings.embed_model = embed_model

In [12]:
import shutil
import os

In [13]:
data_path = './data'

if os.path.isdir(data_path):
  shutil.rmtree(data_path)
  os.makedirs(data_path)

os.makedirs(data_path, exist_ok=True)

about_newton = [
    'Isaac Newton, known for his laws of motion and universal gravitation, laid the groundwork for classical mechanics.',
    'Newton’s work in the 17th century provided the foundation for much of modern physics.'
    ]
with open(data_path + '/newton.txt', mode='w') as f:
  f.writelines(about_newton)

about_edison = [
    'Albert Einstein developed the theory of relativity, which revolutionized theoretical physics and astronomy.',
    'The theory of relativity was formulated in the early 20th century and has had a profound impact on our understanding of space and time.',
    'In 1915, Einstein presented the general theory of relativity, expanding on his earlier work on special relativity.'
]
with open(data_path + '/edison.txt', mode='w') as f:
  f.writelines(about_edison)

documents = SimpleDirectoryReader(data_path).load_data()

In [14]:
# Create index in Neptune database using the test data - automated knowledge graph construction from unstructured text
graph_store = NeptuneDatabaseGraphStore(
    host=neptune_endpoint, port=8182, region_name="us-east-1"
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [15]:
index = KnowledgeGraphIndex.from_documents(
  documents,
  storage_context=storage_context,
  llm=llm,
  show_progress=True,
  include_embeddings=True,
  embed_model=embed_model
)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
# Query
response = index.as_query_engine(llm=llm,).query(
  "How did the scientific contributions of the 17th century influence early 20th-century physics?",
)

In [23]:
response.response

' The scientific discoveries and theories developed during the 17th century laid the groundwork for significant advancements in physics during the early 20th century. The foundational work of Isaac Newton in classical mechanics paved the way for the formulation of more complex theories in later centuries. These theories, in turn, influenced and expanded upon the understanding of physics established during the 17th century. However, without directly referencing the context, it is essential to note that figures like Albert Einstein built upon the knowledge and discoveries of their predecessors to make groundbreaking contributions to the field.'